In [1]:
import folium
import pandas as pd

In [2]:
res=pd.read_csv('res_location.csv',index_col='res')

In [3]:
u_del_cent=pd.read_csv('user_del_cent.csv',index_col='user_id')

In [4]:
u_res_cent=pd.read_csv('user_res_cent.csv',index_col='user_id')

In [5]:
d=[]
for i in u_res_cent.index:
    if i not in u_del_cent.index:
        d.append(i)
        print(len(d),end='\r')
u_res_cent=u_res_cent.drop(d)

In [6]:
def modify(s):
    l=s.split(',')
    p=[float(x[:6]) for x in l[::2]]
    q=[float(x[:6]) for x in l[1::2]]
    r=[]
    for i in range(len(p)):
        r.append([p[i],q[i]])
    return r

In [7]:
for i in range(u_del_cent.shape[0]):
    u_del_cent.iloc[i]['centroid']=modify(u_del_cent.iloc[i]['centroid'])
for i in range(u_res_cent.shape[0]):
    u_res_cent.iloc[i]['centroid']=modify(u_res_cent.iloc[i]['centroid'])

In [8]:
u_del_res=pd.read_csv('user_del_res.csv',index_col='user_id')
u_rev_res=pd.read_csv('user_res_list.csv',index_col='user_id')

In [9]:
for i in range(u_del_res.shape[0]):
    u_del_res.iloc[i]['res_list']=u_del_res.iloc[i]['res_list'].split(',')
for i in range(u_rev_res.shape[0]):
    u_rev_res.iloc[i]['res_list']=u_rev_res.iloc[i]['res_list'].split(',')

In [10]:
q=0
user_del={}
for i in u_del_cent.index:
    tot=[]
    l=u_del_res.loc[i]['res_list']
    for li in l:
        f={}
        if li in res.index:
            f['name']=li
            f['lat']=float(str(res.loc[li]['lat'])[:6])
            f['long']=float(str(res.loc[li]['long'])[:6])
            tot.append(f)
    if len(tot)>0:
        q+=1
        print(q,end='\r')
        user_del[i]=pd.DataFrame(tot)

In [11]:
q=0
user_rev={}
for i in u_res_cent.index:
    tot=[]
    l=u_rev_res.loc[i]['res_list']
    for li in l:
        f={}
        if li in res.index:
            f['name']=li
            f['lat']=float(str(res.loc[li]['lat'])[:6])
            f['long']=float(str(res.loc[li]['long'])[:6])
            tot.append(f)
    if len(tot)>0:
        q+=1
        print(q,end='\r')
        user_rev[i]=pd.DataFrame(tot)

In [14]:
def get_map(uid):
    del_cen=u_del_cent.loc[uid]['centroid']
    #print(del_cen[0])
    res_cen=u_res_cent.loc[uid]['centroid']
    del_data=user_del[uid]
    rev_data=user_rev[uid]
    m=folium.Map(location=del_cen[0])
    fgrr=folium.FeatureGroup(name='Review Restaurants')
    for i in range(rev_data.shape[0]):
        folium.Marker([rev_data.iloc[i]['lat'], rev_data.iloc[i]['long']], popup=rev_data.iloc[i]['name'],icon=folium.Icon(color='blue', icon='info-sign')).add_to(fgrr)
    fgdr=folium.FeatureGroup(name='Delivery Restaurants')
    for i in range(del_data.shape[0]):
        folium.Marker([del_data.iloc[i]['lat'], del_data.iloc[i]['long']], popup=del_data.iloc[i]['name'],icon=folium.Icon(color='black', icon='info-sign')).add_to(fgdr)
    fgdl=folium.FeatureGroup(name='Delivery Locations')
    for i in range(len(del_cen)):
        folium.Marker(del_cen[i],popup='del addr:{}'.format(i+1),icon=folium.Icon(color='red', icon='remove-sign')).add_to(fgdl)
    fgrl=folium.FeatureGroup(name='Eating Locations')
    for i in range(len(res_cen)):
        folium.Marker(res_cen[i],popup='rev addr:{}'.format(i+1),icon=folium.Icon(color='purple', icon='remove-sign')).add_to(fgrl)
    m.add_child(fgrr)
    m.add_child(fgdr)
    m.add_child(fgrl)
    m.add_child(fgdl)
    m.add_child(folium.LayerControl())
    m.save('maps2/{}.html'.format(uid))
    #return m

In [18]:
uids=u_del_cent.index

In [ ]:
for i,uid in enumerate(uids):
    get_map(uid)
    print(i,end='\r')

In [17]:
get_map(1640716)